# Peer-graded Assignment: Capstone Project - The Battle of Neighborhoods (Week 1)
Now that you have been equipped with the skills and the tools to use location data to explore a geographical location, over the course of two weeks, you will have the opportunity to be as creative as you want and come up with an idea to leverage the Foursquare location data to explore or compare neighborhoods or cities of your choice or to come up with a problem that you can use the Foursquare location data to solve.

### 1) Introduction/Business Problem
Clearly define a problem or an idea of your choice, where you would need to leverage the Foursquare location data to solve or execute. Remember that data science problems always target an audience and are meant to help a group of stakeholders solve a problem, so make sure that you explicitly describe your audience and why they would care about your problem.

The idea of this study is to help people planning to open a new restaurant in Toronto to chose the right location by providing data about the income and population of each neighborhood as well as the competitors already present on the same r regions.

### 2) Data
Describe the data that you will be using to solve the problem or execute your idea. Remember that you will need to use the Foursquare location data to solve the problem or execute your idea. You can absolutely use other datasets in combination with the Foursquare location data. So make sure that you provide adequate explanation and discussion, with examples, of the data that you will be using, even if it is only Foursquare location data.

To provide the stakeholders the necessary information I'll be combining Toronto's 2016 Census that contains Population, Average income per Neighborhood with Foursquare API to collect competitors on the same neighborhoods.

Toronto's Census data in publicly available at this website: https://www.toronto.ca/city-government/data-research-maps/open-data/open-data-catalogue/#8c732154-5012-9afe-d0cd-ba3ffc813d5a

With 

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    scipy-1.4.1                |   py36h921218d_0        18.9 MB  conda-forge
    lib

### 2.1) Download and Explore Dataset
Using data from 2016 census to collect Toronto's income per neighborhood

In [125]:
# Toronto Open Data Catalogue - Neighbourhood Profiles 2016 (CSV)
# https://www.toronto.ca/city-government/data-research-maps/open-data/open-data-catalogue/#8c732154-5012-9afe-d0cd-ba3ffc813d5a

csv_path='https://www.toronto.ca/ext/open_data/catalog/data_set_files/2016_neighbourhood_profiles.csv'
df = pd.read_csv(csv_path,encoding='latin1')
print('Data loaded')

Data loaded


In [126]:
df.head()


,Category,Topic,Data Source,Characteristic,City of Toronto,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,Banbury-Don Mills,Bathurst Manor,Bay Street Corridor,Bayview Village,Bayview Woods-Steeles,Bedford Park-Nortown,Beechborough-Greenbrook,Bendale,Birchcliffe-Cliffside,Black Creek,Blake-Jones,Briar Hill-Belgravia,Bridle Path-Sunnybrook-York Mills,Broadview North,Brookhaven-Amesbury,Cabbagetown-South St. James Town,Caledonia-Fairbank,Casa Loma,Centennial Scarborough,Church-Yonge Corridor,Clairlea-Birchmount,Clanton Park,Cliffcrest,Corso Italia-Davenport,Danforth,Danforth East York,Don Valley Village,Dorset Park,Dovercourt-Wallace Emerson-Junction,Downsview-Roding-CFB,Dufferin Grove,East End-Danforth,Edenbridge-Humber Valley,Eglinton East,Elms-Old Rexdale,Englemount-Lawrence,Eringate-Centennial-West Deane,Etobicoke West Mall,Flemingdon Park,Forest Hill North,Forest Hill South,Glenfield-Jane Heights,Greenwood-Coxwell,Guildwood,Henry Farm,High Park North,High Park-Swansea,Highland Creek,Hillcrest Village,Humber Heights-Westmount,Humber Summit,Humbermede,Humewood-Cedarvale,Ionview,Islington-City Centre West,Junction Area,Keelesdale-Eglinton West,Kennedy Park,Kensington-Chinatown,Kingsview Village-The Westway,Kingsway South,Lambton Baby Point,L'Amoreaux,Lansing-Westgate,Lawrence Park North,Lawrence Park South,Leaside-Bennington,Little Portugal,Long Branch,Malvern,Maple Leaf,Markland Wood,Milliken,Mimico (includes Humber Bay Shores),Morningside,Moss Park,Mount Dennis,Mount Olive-Silverstone-Jamestown,Mount Pleasant East,Mount Pleasant West,New Toronto,Newtonbrook East,Newtonbrook West,Niagara,North Riverdale,North St. James Town,Oakridge,Oakwood Village,O'Connor-Parkview,Old East York,Palmerston-Little Italy,Parkwoods-Donalda,Pelmo Park-Humberlea,Playter Estates-Danforth,Pleasant View,Princess-Rosethorn,Regent Park,Rexdale-Kipling,Rockcliffe-Smythe,Roncesvalles,Rosedale-Moore Park,Rouge,Runnymede-Bloor West Village,Rustic,Scarborough Village,South Parkdale,South Riverdale,St.Andrew-Windfields,Steeles,Stonegate-Queensway,Tam O'Shanter-Sullivan,Taylor-Massey,The Beaches,Thistletown-Beaumond Heights,Thorncliffe Park,Trinity-Bellwoods,University,Victoria Village,Waterfront Communities-The Island,West Hill,West Humber-Clairville,Westminster-Branson,Weston,Weston-Pelham Park,Wexford/Maryvale,Willowdale East,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park
0,Neighbourhood Information,Neighbourhood Information,City of Toronto,Neighbourhood Number,NaN,129,128,20,95,42,34,76,52,49,39,112,127,122,24,69,108,41,57,30,71,109,96,133,75,120,33,123,92,66,59,47,126,93,26,83,62,9,138,5,32,11,13,44,102,101,25,65,140,53,88,87,134,48,8,21,22,106,125,14,90,110,124,78,6,15,114,117,38,105,103,56,84,19,132,29,12,130,17,135,73,115,2,99,104,18,50,36,82,68,74,121,107,54,58,80,45,23,67,46,10,72,4,111,86,98,131,89,28,139,85,70,40,116,16,118,61,63,3,55,81,79,43,77,136,1,35,113,91,119,51,37,7,137,64,60,94,100,97,27,31
1,Neighbourhood Information,Neighbourhood Information,City of Toronto,TSNS2020 Designation,NaN,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,NIA,No Designation,No Designation,NIA,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,Emerging Neighbourhood,No Designation,NIA,No Designation,No Designation,No Designation,NIA,NIA,Emerging Neighbourhood,No Designation,No Designation,NIA,No Designation,No Designation,NIA,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,Emerging Neighbourhood,NIA,NIA,No Designation,NIA,No Designation,No Designation,NIA,NIA,No Designat

In [283]:
Neighbourhoods = list(df.columns.values)
Neighbourhoods = Neighbourhoods[5:]
dfToronto = pd.DataFrame(index = Neighbourhoods,columns=["code_id","Population_2016","Income_2016"])
# Population_2016 = Population, 2016
# Income_2016 = Total income: Average amount ($)

for index, row in dfToronto.iterrows():
    dfToronto.at[index, 'code_id'] = df[index][0]
    dfToronto.at[index, 'Population_2016'] = df[index][2]
    dfToronto.at[index, 'Income_2016'] = df[index][2264]

In [285]:
dfToronto["Population_2016"] = list(map(lambda x : x.replace(",",""),dfToronto["Population_2016"]))
dfToronto["Income_2016"] = list(map(lambda x : x.replace(",",""),dfToronto["Income_2016"]))

In [286]:
dfToronto = dfToronto.astype(int)
dfToronto = dfToronto.sort_values(by = "code_id")
dfToronto = dfToronto.reset_index().rename(columns = {"index" : "Neighbourhoods"})
dfToronto.head()

,Neighbourhoods,code_id,Population_2016,Income_2016
0,West Humber-Clairville,1,33312,31771
1,Mount Olive-Silverstone-Jamestown,2,32954,26548
2,Thistletown-Beaumond Heights,3,10360,32815
3,Rexdale-Kipling,4,10529,34418
4,Elms-Old Rexdale,5,9456,32012


,Neighbourhoods,Neighbourhoods,code_id,Population_2016,Income_2016
0,0,West Humber-Clairville,1,33312,31771
1,1,Mount Olive-Silverstone-Jamestown,2,32954,26548
2,2,Thistletown-Beaumond Heights,3,10360,32815
3,3,Rexdale-Kipling,4,10529,34418
4,4,Elms-Old Rexdale,5,9456,32012


In [57]:
# load  long,lat data, link : https://open.toronto.ca/dataset/neighbourhoods/
df_geo = pd.read_csv("Neighbourhoods.csv")
df_geo.head(5)

,_id,AREA_ID,AREA_ATTR_ID,PARENT_AREA_ID,AREA_SHORT_CODE,AREA_LONG_CODE,AREA_NAME,AREA_DESC,X,Y,LONGITUDE,LATITUDE,OBJECTID,Shape__Area,Shape__Length,geometry
0,2801,25886861,25926662,49885,94,94,Wychwood (94),Wychwood (94),NaN,NaN,-79.425515,43.676919,16491505,3.217960e+06,7515.779658,"{u'type': u'Polygon', u'coordinates': (((-79.4..."
1,2802,25886820,25926663,49885,100,100,Yonge-Eglinton (100),Yonge-Eglinton (100),NaN,NaN,-79.403590,43.704689,16491521,3.160334e+06,7872.021074,"{u'type': u'Polygon', u'coordinates': (((-79.4..."
2,2803,25886834,25926664,49885,97,97,Yonge-St.Clair (97),Yonge-St.Clair (97),NaN,NaN,-79.397871,43.687859,16491537,2.222464e+06,8130.411276,"{u'type': u'Polygon', u'coordinates': (((-79.3..."
3,2804,25886593,25926665,49885,27,27,York University Heights (27),York University Heights (27),NaN,NaN,-79.488883,43.765736,16491553,2.541821e+07,25632.335242,"{u'type': u'Polygon', u'coordinates': (((-79.5..."
4,2805,25886688,25926666,49885,31,31,Yorkdale-Glen Park (31),Yorkdale-Glen Park (31),NaN,NaN,-79.457108,43.714672,16491569,1.156669e+07,13953.408098,"{u'type': u'Polygon', u'coordinates': (((-79.4..."


In [71]:
df_geo_toronto = pd.DataFrame(columns=["code_id","Longitude","Latitude"])
df_geo_toronto["code_id"] = df_geo["AREA_SHORT_CODE"]
df_geo_toronto["Longitude"] = df_geo["LONGITUDE"]
df_geo_toronto["Latitude"] = df_geo["LATITUDE"]

In [90]:
df_geo_toronto = df_geo_toronto.sort_values(by = "code_id")
df_geo_toronto.head()

,code_id,Longitude,Latitude
63,1,-79.596356,43.716180
20,2,-79.587259,43.746868
56,3,-79.563491,43.737988
40,4,-79.566228,43.723725
112,5,-79.548983,43.721519


### merge data 

In [155]:
df_new = pd.merge(dfToronto, df_geo_toronto[["code_id","Longitude","Latitude"]], on = "code_id")

In [156]:
df_new.to_csv("Toronto.csv")
df_new.head()

,Neighbourhoods,code_id,Population_2016,Income_2016,Longitude,Latitude
0,West Humber-Clairville,1,33312,31771,-79.596356,43.716180
1,Mount Olive-Silverstone-Jamestown,2,32954,26548,-79.587259,43.746868
2,Thistletown-Beaumond Heights,3,10360,32815,-79.563491,43.737988
3,Rexdale-Kipling,4,10529,34418,-79.566228,43.723725
4,Elms-Old Rexdale,5,9456,32012,-79.548983,43.721519


In [117]:
!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [118]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [159]:
map_Toranto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(df_new['Latitude'], df_new['Longitude'], df_new['Neighbourhoods']):
    label = "{}".format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toranto)  
    
map_Toranto

In [160]:
CLIENT_ID = '00UX0ISEZI0YPT5NF3HENXKV50IIO1E4OHS4WK4Y0EEY5QZA' # your Foursquare ID
CLIENT_SECRET = 'QYHPJNHQWWSIQ5LNXX5GUYB1L4LIUQTV3BHXHJESLOUU2HKF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 
radius = 500
latitude = df_new.loc[0,'Latitude'] 
longitude = df_new.loc[0,'Longitude']
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 00UX0ISEZI0YPT5NF3HENXKV50IIO1E4OHS4WK4Y0EEY5QZA
CLIENT_SECRET:QYHPJNHQWWSIQ5LNXX5GUYB1L4LIUQTV3BHXHJESLOUU2HKF


In [161]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhoods', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [165]:
import requests # Library for web scraping
names = df_new['Neighbourhoods']
latitudes = df_new['Latitude']
longitudes = df_new['Longitude']
toronto_venues = getNearbyVenues(names,latitudes,longitudes)

West Humber-Clairville
Mount Olive-Silverstone-Jamestown
Thistletown-Beaumond Heights
Rexdale-Kipling
Elms-Old Rexdale
Kingsview Village-The Westway
Willowridge-Martingrove-Richview
Humber Heights-Westmount
Edenbridge-Humber Valley
Princess-Rosethorn
Eringate-Centennial-West Deane
Markland Wood
Etobicoke West Mall
Islington-City Centre West
Kingsway South
Stonegate-Queensway
Mimico (includes Humber Bay Shores)
New Toronto
Long Branch
Alderwood
Humber Summit
Humbermede
Pelmo Park-Humberlea
Black Creek
Glenfield-Jane Heights
Downsview-Roding-CFB
York University Heights
Rustic
Maple Leaf
Brookhaven-Amesbury
Yorkdale-Glen Park
Englemount-Lawrence
Clanton Park
Bathurst Manor
Westminster-Branson
Newtonbrook West
Willowdale West
Lansing-Westgate
Bedford Park-Nortown
St.Andrew-Windfields
Bridle Path-Sunnybrook-York Mills
Banbury-Don Mills
Victoria Village
Flemingdon Park
Parkwoods-Donalda
Pleasant View
Don Valley Village
Hillcrest Village
Bayview Woods-Steeles
Newtonbrook East
Willowdale East


In [262]:
#b = [i if "Restaurant" in i else 0 for i in toronto_venues["Venue Category"] ]
#b
toronto_venues["check"] = [i if "Restaurant" in i else float("nan") for i in toronto_venues["Venue Category"] ]
toronto_venues.dropna(axis=0,inplace = True)
toronto_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,check
2,West Humber-Clairville,43.716180,-79.596356,Mandarin Buffet,43.720360,-79.594387,Chinese Restaurant,Chinese Restaurant
3,West Humber-Clairville,43.716180,-79.596356,Xawaash,43.715786,-79.593053,Mediterranean Restaurant,Mediterranean Restaurant
7,West Humber-Clairville,43.716180,-79.596356,Swiss Pick,43.716150,-79.593843,Swiss Restaurant,Swiss Restaurant
17,Mount Olive-Silverstone-Jamestown,43.746868,-79.587259,NORI SUSHI,43.742775,-79.586985,Japanese Restaurant,Japanese Restaurant
18,Thistletown-Beaumond Heights,43.737988,-79.563491,Drupati's Doubles & Roti Shop,43.736222,-79.563471,Caribbean Restaurant,Caribbean Restaurant


In [273]:
toronto_venues.groupby('Neighbourhood').count()
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
#toronto_onehot.drop(['Neighbourhood'],axis=1,inplace=True) 
toronto_onehot.insert(loc=0, column='Neighbourhood', value=toronto_venues['Neighbourhood'] )
toronto_onehot.head()

,Neighbourhood,African Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Belgian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Comfort Food Restaurant,Cuban Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Greek Restaurant,Halal Restaurant,Hotpot Restaurant,Indian Chinese Restaurant,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,New American Restaurant,North Indian Restaurant,Pakistani Restaurant,Persian Restaurant,Portuguese Restaurant,Ramen Restaurant,Restaurant,Seafood Restaurant,South American Restaurant,South Indian Restaurant,Sushi Restaurant,Swiss Restaurant,Syrian Restaurant,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Tibetan Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
2,West Humber-Clairville,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,West Humber-Clairville,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,West Humber-Clairville,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
17,Mount Olive-Silverstone-Jamestown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18,Thistletown-Beaumond Heights,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [278]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.shape

(90, 58)

In [280]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [281]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt North,Chinese Restaurant,Japanese Restaurant,Fast Food Restaurant,Dim Sum Restaurant,Vietnamese Restaurant,Egyptian Restaurant,Indian Chinese Restaurant,Hotpot Restaurant,Halal Restaurant,Greek Restaurant
1,Agincourt South-Malvern West,Chinese Restaurant,Malay Restaurant,Mediterranean Restaurant,American Restaurant,Asian Restaurant,Restaurant,Cantonese Restaurant,Seafood Restaurant,Falafel Restaurant,Fast Food Restaurant
2,Annex,Vegetarian / Vegan Restaurant,American Restaurant,French Restaurant,Middle Eastern Restaurant,Indian Restaurant,Egyptian Restaurant,Indian Chinese Restaurant,Hotpot Restaurant,Halal Restaurant,Greek Restaurant
3,Banbury-Don Mills,Italian Restaurant,Cantonese Restaurant,Eastern European Restaurant,Indian Chinese Restaurant,Hotpot Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Filipino Restaurant
4,Bay Street Corridor,Italian Restaurant,Chinese Restaurant,Fast Food Restaurant,Japanese Restaurant,Sushi Restaurant,Restaurant,Ramen Restaurant,American Restaurant,Caribbean Restaurant,Falafel Restaurant


In [287]:
# set number of clusters
dfToronto = dfToronto.drop("code_id",axis =1).rename(columns = {"Neighbourhoods" :"Neighbourhood"})
toronto_grouped = pd.merge(dfToronto, toronto_grouped, on = "Neighbourhood")
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([4, 4, 4, 4, 3, 0, 0, 0, 0, 0, 4, 4, 4, 3, 2, 0, 4, 4, 4, 4, 4, 4,
       4, 4, 3, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 4, 0, 4, 0, 0, 4, 0, 0,
       0, 0, 4, 0, 4, 0, 0, 0, 0, 4, 4, 4, 0, 3, 1, 3, 3, 3, 3, 1, 0, 3,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4], dtype=int32)

In [288]:

# add clustering labels
df_new = df_new.rename(columns = {"Neighbourhoods" :"Neighbourhood"})
neighbourhoods_venues_sorted["Cluster_Labels"] = kmeans.labels_

toronto_merged = df_new

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() 

,Neighbourhood,code_id,Population_2016,Income_2016,Longitude,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster_Labels
0,West Humber-Clairville,1,33312,31771,-79.596356,43.716180,Mediterranean Restaurant,Swiss Restaurant,Chinese Restaurant,Vietnamese Restaurant,Eastern European Restaurant,Hotpot Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,French Restaurant,4.0
1,Mount Olive-Silverstone-Jamestown,2,32954,26548,-79.587259,43.746868,Japanese Restaurant,Vietnamese Restaurant,Eastern European Restaurant,Indian Chinese Restaurant,Hotpot Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,0.0
2,Thistletown-Beaumond Heights,3,10360,32815,-79.563491,43.737988,Indian Restaurant,Caribbean Restaurant,Thai Restaurant,Asian Restaurant,Eastern European Restaurant,Indian Chinese Restaurant,Hotpot Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,4.0
3,Rexdale-Kipling,4,10529,34418,-79.566228,43.723725,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Elms-Old Rexdale,5,9456,32012,-79.548983,43.721519,African Restaurant,Caribbean Restaurant,Eastern European Restaurant,Indian Chinese Restaurant,Hotpot Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,0.0


In [289]:
toronto_merged=toronto_merged.dropna()
toronto_merged['Cluster_Labels']=toronto_merged['Cluster_Labels'].astype(int)

In [290]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [291]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 0, toronto_merged.columns[[1] + list(range(0, toronto_merged.shape[1]))]]

,code_id,Neighbourhood,code_id,Population_2016,Income_2016,Longitude,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster_Labels
1,2,Mount Olive-Silverstone-Jamestown,2,32954,26548,-79.587259,43.746868,Japanese Restaurant,Vietnamese Restaurant,Eastern European Restaurant,Indian Chinese Restaurant,Hotpot Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,0
4,5,Elms-Old Rexdale,5,9456,32012,-79.548983,43.721519,African Restaurant,Caribbean Restaurant,Eastern European Restaurant,Indian Chinese Restaurant,Hotpot Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,0
10,11,Eringate-Centennial-West Deane,11,18588,47002,-79.580445,43.658017,Chinese Restaurant,Vietnamese Restaurant,Eastern European Restaurant,Indian Chinese Restaurant,Hotpot Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,0
11,12,Markland Wood,12,10554,62378,-79.573432,43.633542,Fast Food Restaurant,Vietnamese Restaurant,Indonesian Restaurant,Indian Chinese Restaurant,Hotpot Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,0
13,14,Islington-City Centre West,14,43965,52787,-79.543317,43.633463,Fast Food Restaurant,Vietnamese Restaurant,Turkish Restaurant,Thai Restaurant,Greek Restaurant,Restaurant,Dumpling Restaurant,Hotpot Restaurant,Halal Restaurant,German Restaurant,0
17,18,New Toronto,18,11463,44101,-79.510358,43.600589,Italian Restaurant,Falafel Restaurant,Indian Restaurant,Argentinian Restaurant,Egyptian Restaurant,Indian Chinese Restaurant,Hotpot Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,0
18,19,Long Branch,19,10084,47384,-79.533345,43.592362,Italian Restaurant,Greek Restaurant,Restaurant,Dumpling Restaurant,Indian Chinese Restaurant,Hotpot Restaurant,Halal Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,0
28,29,Maple Leaf,29,10111,37108,-79.480758,43.715574,Chinese Restaurant,Vietnamese Restaurant,Eastern European Restaurant,Indian Chinese Restaurant,Hotpot Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,0
38,39,Bedford Park-Nortown,39,23236,123077,-79.420227,43.731486,Italian Restaurant,Fast Food Restaurant,American Restaurant,Thai Restaurant,Comfort Food Restaurant,Restaurant,Sushi Restaurant,Filipino Restaurant,Ethiopian Restaurant,Falafel Restaurant,0
43,44,Flemingdon Park,44,21933,28654,-79.332646,43.715930,Japanese Restaurant,Middle Eastern Restaurant,Chinese Restaurant,Fast Food Restaurant,Falafel Restaurant,Pakistani Restaurant,Vietnamese Restaurant,Egyptian Restaurant,Hotpot Restaurant,Halal Restaurant,0


In [221]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 1, toronto_merged.columns[[1] + list(range(0, toronto_merged.shape[1]))]]

,code_id,Neighbourhood,code_id,Population_2016,Income_2016,Longitude,Latitude,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,4,Rexdale-Kipling,4,10529,34418,-79.566228,43.723725,1,Flower Shop,Zoo Exhibit,Falafel Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Farm
9,10,Princess-Rosethorn,10,11051,99055,-79.544559,43.666051,1,Playground,Athletics & Sports,Zoo Exhibit,Falafel Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Farm
22,23,Pelmo Park-Humberlea,23,10722,39003,-79.528282,43.717515,1,Bakery,Auto Garage,IT Services,Furniture / Home Store,Falafel Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service
68,69,Blake-Jones,69,7727,48511,-79.337394,43.676173,1,Café,Coffee Shop,Convenience Store,Dim Sum Restaurant,Beer Bar,Toy / Game Store,Bookstore,Asian Restaurant,Gastropub,Diner
79,80,Palmerston-Little Italy,80,13826,58071,-79.418409,43.659157,1,Karaoke Bar,Sandwich Place,Café,Dessert Shop,Coffee Shop,Nightclub,Japanese Restaurant,Music Store,Gay Bar,Indian Restaurant


In [222]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 2, toronto_merged.columns[[1] + list(range(0, toronto_merged.shape[1]))]]

,code_id,Neighbourhood,code_id,Population_2016,Income_2016,Longitude,Latitude,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,West Humber-Clairville,1,33312,31771,-79.596356,43.716180,2,Department Store,Hotel,Mediterranean Restaurant,Bank,Racecourse,Coffee Shop,Paper / Office Supplies Store,Chinese Restaurant,Electronics Store,Shopping Mall
2,3,Thistletown-Beaumond Heights,3,10360,32815,-79.563491,43.737988,2,Indian Restaurant,Caribbean Restaurant,ATM,Asian Restaurant,Pharmacy,Coffee Shop,Spa,Supermarket,Thai Restaurant,Bank
6,7,Willowridge-Martingrove-Richview,7,22156,44177,-79.554221,43.683645,2,Gym,Zoo Exhibit,Donut Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Falafel Restaurant
10,11,Eringate-Centennial-West Deane,11,18588,47002,-79.580445,43.658017,2,Hockey Arena,Pizza Place,Chinese Restaurant,Park,Coffee Shop,Convenience Store,Curling Ice,Dance Studio,Fish & Chips Shop,Filipino Restaurant
11,12,Markland Wood,12,10554,62378,-79.573432,43.633542,2,Bank,Pizza Place,Discount Store,Fast Food Restaurant,Falafel Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space
12,13,Etobicoke West Mall,13,11848,38255,-79.568939,43.645063,2,Farmers Market,IT Services,Transportation Service,Coffee Shop,Sandwich Place,Zoo Exhibit,Event Service,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant
13,14,Islington-City Centre West,14,43965,52787,-79.543317,43.633463,2,Fast Food Restaurant,Sandwich Place,Turkish Restaurant,Fried Chicken Joint,Café,Thai Restaurant,Greek Restaurant,Ice Cream Shop,Pizza Place,Restaurant
19,20,Alderwood,20,12054,47709,-79.541611,43.604937,2,Pizza Place,Convenience Store,Pharmacy,Coffee Shop,Athletics & Sports,Event Space,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant
25,26,Downsview-Roding-CFB,26,35052,34168,-79.490497,43.733292,2,Park,Moving Target,Business Service,Zoo Exhibit,Event Space,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Falafel Restaurant
26,27,York University Heights,27,27593,29958,-79.488883,43.765736,2,Coffee Shop,Fast Food Restaurant,Caribbean Restaurant,Japanese Restaurant,Falafel Restaurant,Bar,Bank,Massage Studio,Pizza Place,Field


In [223]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 3, toronto_merged.columns[[1] + list(range(0, toronto_merged.shape[1]))]]

,code_id,Neighbourhood,code_id,Population_2016,Income_2016,Longitude,Latitude,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
137,138,Eglinton East,138,22776,30033,-79.245598,43.740922,3,Pizza Place,Bowling Alley,Japanese Restaurant,Rock Club,Grocery Store,Zoo Exhibit,Event Space,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant


In [224]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 4, toronto_merged.columns[[1] + list(range(0, toronto_merged.shape[1]))]]

,code_id,Neighbourhood,code_id,Population_2016,Income_2016,Longitude,Latitude,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,9,Edenbridge-Humber Valley,9,15535,101551,-79.522458,43.670886,4,Dog Run,Park,Fast Food Restaurant,Event Space,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Falafel Restaurant
38,39,Bedford Park-Nortown,39,23236,123077,-79.420227,43.731486,4,Coffee Shop,Italian Restaurant,Sandwich Place,Fast Food Restaurant,Liquor Store,Bagel Shop,Pizza Place,Comfort Food Restaurant,Pub,Restaurant
44,45,Parkwoods-Donalda,45,34805,42516,-79.330180,43.755033,4,Pool,Food & Drink Shop,Park,Event Service,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space
45,46,Pleasant View,46,15818,36346,-79.334948,43.786982,4,Thrift / Vintage Store,Restaurant,Burger Joint,Breakfast Spot,Fast Food Restaurant,Skating Rink,Shopping Mall,Bakery,Japanese Restaurant,Pizza Place
47,48,Hillcrest Village,48,16934,40442,-79.354804,43.802988,4,Residential Building (Apartment / Condo),Park,Donut Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Zoo Exhibit
48,49,Bayview Woods-Steeles,49,13154,47246,-79.382118,43.796802,4,Dog Run,Park,Basketball Court,Zoo Exhibit,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Falafel Restaurant
57,58,Old East York,58,9233,53315,-79.335488,43.696781,4,Japanese Restaurant,Pastry Shop,Grocery Store,Restaurant,Zoo Exhibit,Dog Run,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store
66,67,Playter Estates-Danforth,67,7804,70831,-79.354887,43.679700,4,Greek Restaurant,Pub,Yoga Studio,Coffee Shop,Ice Cream Shop,Italian Restaurant,Spa,Dance Studio,Juice Bar,Tapas Restaurant
71,72,Regent Park,72,10803,34597,-79.360509,43.659992,4,Coffee Shop,Restaurant,Thai Restaurant,Pub,Electronics Store,Café,Food Truck,Sushi Restaurant,Beer Store,Indian Restaurant
85,86,Roncesvalles,86,14974,50580,-79.442992,43.646123,4,Coffee Shop,Pizza Place,Park,American Restaurant,Eastern European Restaurant,Food & Drink Shop,Grocery Store,Thai Restaurant,Toy / Game Store,Pharmacy
